In [109]:
from pymongo import MongoClient,TEXT
import matplotlib.pyplot as plt
import urllib.parse
import pandas as pd
import urllib.robotparser
import plotly.express as px
import plotly.graph_objects as go
from wordcloud import WordCloud

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/linuxperso/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [134]:
client=MongoClient('localhost', port=27017)
performance={}
list_match="trump"#"france"#"climat"#"cop.?28"
database="scrapping"
collection_sitemaps="sitemaps"
collection_htmls="htmls"


print("\nNb pages html :{}".format(client[database][collection_htmls].count_documents({})))


Nb pages html :13448825


## Requête avec uniquement la Regex

In [111]:
import time


t1=time.time()

def match_document_v1(client, database, collection_htmls, list_match):
    return pd.DataFrame(list(client[database][collection_htmls].aggregate(
                                                        [
                                                            {"$match" : 
                                                                {
                                                                    "$and":
                                                                        [
                                                                            {
                                                                                "url" : # doit contenir la liste de mots clés
                                                                                        { 
                                                                                            '$regex' : '^(.*({list_match})).*$'.format(list_match=list_match),
                                                                                            '$options' : 'i'
                                                                                        },  
                                                                            }
                                                                        ]
                                                                }
                                                            },
                                                            { "$project": 
                                                                    { 
                                                                        "_id":0,
                                                                        "id_media": 1, 
                                                                        "media_name": 1, 
                                                                        "url" : 1,
                                                                        "mots_in_url" :1
                                                                    }
                                                            }
                                                        ]
                                                )
                                        )
)
df_urls_match_v1=match_document_v1(client, database, collection_htmls, list_match)
t2=time.time()
print(t2-t1)
performance["regex_only"]=[len(df_urls_match_v1),t2-t1]


70.30713748931885


In [112]:
df_urls_match_v1

,url,id_media,media_name
0,https://abcnews.go.com/alerts/donald-trump,2581,ABC
1,https://www.nationalreview.com/2024/01/trumps-...,2812,National Review
2,https://i0.wp.com/www.nationalreview.com/wp-co...,2812,National Review
3,https://i0.wp.com/www.nationalreview.com/wp-co...,2812,National Review
4,https://www.theguardian.com/us-news/2024/jan/0...,2847,Guardian
...,...,...,...
303346,https://www.msnbc.com/rachel-maddow-show/after...,2587,MSNBC
303347,https://www.msnbc.com/rachel-maddow-show/trump...,2587,MSNBC
303348,https://media-cldnry.s-nbcnews.com/image/uploa...,2587,MSNBC
303349,https://www.msnbc.com/rachel-maddow-show/trump...,2587,MSNBC


# Requete avec uniquement le match sur les elemtn de l'array parsé

In [113]:
t1=time.time()
def match_document_v2(client, database, collection_htmls, list_match):
    return pd.DataFrame(list(client[database][collection_htmls].aggregate(
                                                        [
                                                            {
                                                                "$match" : 
                                                                {
                                                                    "mots_in_url":list_match
                                                                },
                                                            },
                                                            { "$project": 
                                                                    { 
                                                                        "_id":0,
                                                                        "id_media": 1, 
                                                                        "media_name": 1, 
                                                                        "url" : 1, 
                                                                        "mots_in_url" :1
                                                                    }
                                                            }
                                                        ]
                                                )
                                        )
)
df_urls_match_v2=match_document_v2(client, database, collection_htmls, list_match)
t2=time.time()
print(t2-t1)
performance["match_sur_element_only"]=[len(df_urls_match_v2),t2-t1]

6.798863649368286


In [114]:
df_urls_match_v2

""


In [115]:
fusion=df_urls_match_v1.merge(df_urls_match_v2, how="outer", on=['url','url'])
fusion.loc[fusion.isna().any(axis=1)]

KeyError: 'url'

# Requête avec création d'un index de text

In [127]:
if "url_text" in client[database][collection_htmls].index_information():
    client[database][collection_htmls].drop_index('url_text')

client[database][collection_htmls].create_index([('mots_in_url', TEXT)], default_language='english')

'mots_in_url_text'

In [125]:
t1=time.time()
def match_document_v3(client, database, collection_htmls, list_match):
    return pd.DataFrame(list(client[database][collection_htmls].aggregate(
                                                        [
                                                            {
                                                                "$match" : 
                                                                { "$text": { "$search": list_match } } 
                                                            },
                                                            { "$project": 
                                                                    { 
                                                                        "_id":0,
                                                                        "id_media": 1, 
                                                                        "media_name": 1, 
                                                                        "url" : 1, 
                                                                        
                                                                    }
                                                            }
                                                        ]
                                                )
                                        )
)
df_urls_match_v3=match_document_v3(client, database, collection_htmls, list_match)
t2=time.time()
print(t2-t1)
performance["match_sur_element_only_index_text"]=[len(df_urls_match_v3),t2-t1]

0.0029931068420410156


In [126]:
df_urls_match_v3


""


In [119]:
fusion=df_urls_match_v1.merge(df_urls_match_v3, how="outer", on=['url','url'])
fusion.loc[fusion.isna().any(axis=1)]

KeyError: 'url'

# Requête avec index de texte et regex

In [ ]:
t1=time.time()
def match_document_v4(client, database, collection_htmls, list_match):
    return pd.DataFrame(list(client[database][collection_htmls].aggregate(
                                                        [
                                                            {
                                                                "$match" : 
                                                                {
                                                                    "$and":
                                                                            [
                                                                                { "$text": { "$search": list_match } },
                                                                                {
                                                                                "url" : # doit contenir la liste de mots clés
                                                                                        { 
                                                                                            '$regex' : '^(.*({list_match})).*$'.format(list_match=list_match),
                                                                                            '$options' : 'i'
                                                                                        },  
                                                                                }
                                                                            ]
                                                                },
                                                            },
                                                            { "$project": 
                                                                    { 
                                                                        "_id":0,
                                                                        "id_media": 1, 
                                                                        "media_name": 1, 
                                                                        "url" : 1, 
                                                                        "mots_in_url" :1
                                                                    }
                                                            }
                                                        ]
                                                )
                                        )
)
df_urls_match_v4=match_document_v4(client, database, collection_htmls, list_match)
t2=time.time()
print(t2-t1)
performance["match_sur_element_et_regex_with_index_text"]=[len(df_urls_match_v4),t2-t1]

0.023924589157104492


In [ ]:
df_urls_match_v4

,url,mots_in_url,id_media,media_name
0,https://onlinejournal.com/Archive/Media/media....,"[archive, media, media]",2714,Online Journal
1,https://www.petworthnews.org/blog/tag/media,"[blog, tag, media]",2940,Petworth News
2,https://onlinejournal.com/Archive/Media/Solomo...,"[archive, media, solomon072600, solomon072600]",2714,Online Journal
3,https://onlinejournal.com/Archive/Media/Johnso...,"[archive, media, johnson101100, johnson101100]",2714,Online Journal
4,https://onlinejournal.com/Archive/Media/Binion...,"[archive, media, binion121600, binion121600]",2714,Online Journal
...,...,...,...,...
530,https://www.businessinsider.com/lapse-hyped-so...,"[apse, hyped, social, media, startup, raising,...",2602,Business Insider
531,https://www.businessinsider.com/how-travel-ful...,"[ow, travel, full, time, make, money, social, ...",2602,Business Insider
532,https://benzinga.com/sec/insider-trades/azpn/c...,"[sec, insider, trades, azpn, cv, communicatins...",2599,Benzinga
533,https://benzinga.com/sec/insider-trades/imn/sa...,"[sec, insider, trades, imn, sa, europe, media,...",2599,Benzinga


In [ ]:
client[database][collection_htmls].index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)]},
 'mots_in_url_text': {'v': 2,
  'key': [('_fts', 'text'), ('_ftsx', 1)],
  'weights': SON([('mots_in_url', 1)]),
  'default_language': 'english',
  'language_override': 'language',
  'textIndexVersion': 3}}

In [128]:
client[database][collection_htmls].drop_index('mots_in_url_text')

In [129]:
client[database][collection_htmls].create_index([('url', TEXT)], default_language='english')

'url_text'

In [130]:
t1=time.time()
def match_document_v5(client, database, collection_htmls, list_match):
    return pd.DataFrame(list(client[database][collection_htmls].aggregate(
                                                        [
                                                            {
                                                                "$match" : 
                                                                { "$text": { "$search": list_match } } 
                                                            },
                                                            { "$project": 
                                                                    { 
                                                                        "_id":0,
                                                                        "id_media": 1, 
                                                                        "media_name": 1, 
                                                                        "url" : 1, 
                                                                        "mots_in_url" :1
                                                                    }
                                                            }
                                                        ]
                                                )
                                        )
)
df_urls_match_v5=match_document_v3(client, database, collection_htmls, list_match)
t2=time.time()
performance["match_sur_url_with_index_text"]=[len(df_urls_match_v5),t2-t1]
print(t2-t1)

2.266772985458374


In [131]:
df_urls_match_v5

,url,id_media,media_name
0,https://mashable.com/article/trump-trump-trump...,2686,Mashable
1,https://mashable.com/article/trump-trump-trump...,2686,Mashable
2,https://www.opednews.com/Quicklink/Donald-Trum...,2715,Op Ed News
3,https://www.opednews.com/articles/Why-Does-Tru...,2715,Op Ed News
4,https://www.opednews.com/articles/Trump-s-War-...,2715,Op Ed News
...,...,...,...
292443,https://img.thedailybeast.com/image/upload/c_c...,2658,Daily Beast
292444,https://img.thedailybeast.com/image/upload/c_c...,2658,Daily Beast
292445,https://img.thedailybeast.com/image/upload/c_c...,2658,Daily Beast
292446,https://www.csmonitor.com/Commentary/Global-Ne...,2805,Christian Science Monitor


In [135]:
fusion=df_urls_match_v1.merge(df_urls_match_v5, how="outer", on=['url','url'])
list(fusion.loc[fusion.isna().any(axis=1)].loc[:, "url"])

['https://i0.wp.com/www.nationalreview.com/wp-content/uploads/2024/01/JoeBiden-DonaldTrump-2.jpg?resize=150%2C150&ssl=1',
 'https://talkingpointsmemo.com/edblog/always-attack-some-thoughts-on-politics-in-a-time-of-trumpism',
 'https://talkingpointsmemo.com/edblog/the-saudi-russian-trumpy-alliance',
 'https://talkingpointsmemo.com/newsletter/vol-2-no-60-are-trumpite-leaders-all-crooks',
 'https://truthout.org/articles/the-heritage-foundation-is-preparing-the-ground-for-trumpism-to-seize-the-state/',
 'https://static01.nyt.com/images/2024/01/06/opinion/06trump-editorial/06trump-editorial-articleLarge-v2.jpg',
 'https://static01.nyt.com/images/2024/01/05/multimedia/05trump-trial-tvlw/05trump-trial-tvlw-articleLarge.jpg',
 'https://static01.nyt.com/images/2024/01/04/multimedia/04trump-on-trial-nl-top/04trump-on-trial-nl-top-articleLarge-v3.jpg',
 'https://capitolhillcorner.org/2022/07/07/july-8-friday-night-jazz-at-em-metro-features-paul-bailey-on-trumpet/',
 'https://capitolhillcorner.org

In [133]:
pd.DataFrame.from_dict(performance, orient="index", columns=["nb_resultats", "time"])

,nb_resultats,time
regex_only,303351,70.307137
match_sur_element_only,0,6.798864
match_sur_element_only_index_text,0,0.002993
match_sur_url_with_index_text,292448,2.266773
